In [1]:
import os
import sys
from keras.models import Model
from keras.layers import concatenate
from keras.utils.data_utils import get_file
from keras.optimizers import RMSprop
from data_generator.config import mpii_sp_dataconf
from data_generator.mpii import MpiiSinglePerson
from data_generator.loader import BatchLoader
from models.losses import pose_regression_loss
from models import reception
from models.mpii_tools import eval_singleperson_pckh
from models.mpii_tools import MpiiEvalCallback
from keras.callbacks import LearningRateScheduler
from models.callbacks import SaveModel
from data_generator.io_operate import *
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session


Using TensorFlow backend.


In [2]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))
num_blocks = 8
batch_size = 12
input_shape = mpii_sp_dataconf.input_shape
num_joints = 16
dataset_path = './data/mpii/'
TEST_MODE = 0
TRAIN_MODE = 1
VALID_MODE = 2

In [3]:
mpii = MpiiSinglePerson(dataset_path, dataconf=mpii_sp_dataconf)
mpii_tr = BatchLoader(mpii, ['frame'], ['pose'], TRAIN_MODE,batch_size=batch_size, num_predictions=num_blocks, shuffle=True)
mpii_val = BatchLoader(mpii, x_dictkeys=['frame'],y_dictkeys=['pose', 'afmat', 'headsize'], mode=VALID_MODE,
        batch_size=mpii.get_length(VALID_MODE), num_predictions=1,shuffle=False)
[x_val], [p_val, afmat_val, head_val] = mpii_val[0]

In [4]:
model = reception.build(input_shape, num_joints, dim=2,num_blocks=num_blocks, num_context_per_joint=2, ksize=(5, 5))
logdir = './'
eval_callback = MpiiEvalCallback(x_val, p_val, afmat_val, head_val,eval_model=model, batch_size=2, pred_per_block=1, logdir=logdir)
loss = pose_regression_loss('l1l2bincross', 0.01)
model.compile(loss=loss, optimizer=RMSprop())
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
Stem (Model)                    (None, 32, 32, 576)  1039488     input_1[0][0]                    
__________________________________________________________________________________________________
rBlock1 (Model)                 (None, 32, 32, 576)  1312128     Stem[1][0]                       
__________________________________________________________________________________________________
SepConv1 (Model)                (None, 32, 32, 576)  347904      rBlock1[1][0]                    
__________________________________________________________________________________________________
RegMap1 (M

In [5]:
def lr_scheduler(epoch, lr):

    if epoch in [80, 100]:
        newlr = 0.2*lr
        printcn(WARNING, 'lr_scheduler: lr %g -> %g @ %d' % (lr, newlr, epoch))
    else:
        newlr = lr
        printcn(OKBLUE, 'lr_scheduler: lr %g @ %d' % (newlr, epoch))

    return newlr
weights_file = 'weights_mpii_{epoch:03d}.h5'
callbacks = []
callbacks.append(SaveModel(weights_file))
callbacks.append(LearningRateScheduler(lr_scheduler))
callbacks.append(eval_callback)

steps_per_epoch = mpii.get_length(TRAIN_MODE) // batch_size

model.fit_generator(mpii_tr,
        steps_per_epoch=steps_per_epoch,
        epochs=120,
        callbacks=callbacks,
        workers=4,
        initial_epoch=0)


Epoch 1/120
lr_scheduler: lr 0.001 @ 0
8/8 [==============================] - 85s 11s/step - loss: 1.0697 - concatenate_7_loss: 0.1271 - concatenate_8_loss: 0.1291 - concatenate_9_loss: 0.1320 - concatenate_10_loss: 0.1326 - concatenate_11_loss: 0.1353 - concatenate_12_loss: 0.1360 - concatenate_13_loss: 0.1368 - concatenate_14_loss: 0.1407
Saving model @epoch=00001 to weights_mpii_001.h5
22/22 [==============================] - 6s 260ms/step
PCKh on validation: 6.1 5.8 7.2 4.8 4.4 5.8 6.5 5.8
pelvis | thorax | neck   | head   | r_shoul| l_shoul| r_elb  | l_elb  | r_wrist| l_wrist| r_hip  | l_hip  | r_knww | l_knee | r_ankle| l_ankle| 
 9.09 |  13.64 |  13.64 |  4.55 |  4.55 |  9.09 |  4.55 |  13.64 |  0.00 |  9.09 |  9.09 |  9.09 |  0.00 |  0.00 |  0.00 |  0.00 | 
Best score is 7.2 at epoch 1
Epoch 2/120
lr_scheduler: lr 0.001 @ 1
8/8 [==============================] - 70s 9s/step - loss: 0.8234 - concatenate_7_loss: 0.1023 - concatenate_8_loss: 0.1012 - concatenate_9_loss: 0.1026 - c

KeyboardInterrupt: 

In [3]:
model = reception.build(input_shape, num_joints, dim=2,num_blocks=num_blocks, num_context_per_joint=2, ksize=(5, 5))

In [4]:
weight_path = './weights_mpii_120.h5'
model.load_weights(weight_path)

ValueError: You are trying to load a weight file containing 1 layers into a model with 35 layers.